In [34]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
from citipy import citipy
from config import api_key
from datetime import datetime

In [35]:
# Create a set of random latitude and longitude combinations.
# keep size at 1500
lats = np.random.uniform(low=-90.000, high=90.000, size=20)
lngs = np.random.uniform(low=-180.000, high=180.000, size=20)
lat_lngs = zip(lats, lngs)
#print(lat_lngs)

In [36]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)
coordinates

[(-38.077354131332605, -46.87030485027449),
 (66.42113978537063, 172.54774935338162),
 (21.767651915662952, -22.056099716914105),
 (-63.756992222114064, -134.24473898531286),
 (88.64074061382397, -133.54864507232844),
 (72.0763680057571, -40.35161111254013),
 (-59.903318115512306, -89.1991436705849),
 (-21.706709954073744, -130.0883943255932),
 (-11.520685332815845, 136.58695800674042),
 (70.4504914673438, 66.39548028193857),
 (2.713487257409355, -161.17454714163637),
 (-68.4268113715139, 175.2249060550883),
 (-30.550650591906766, -167.21030630231047),
 (-63.458260955250026, 171.50262647558492),
 (5.078466393817465, -103.97364494262784),
 (26.54811244574475, -91.3886148880499),
 (-20.120054751032782, -21.280555535023552),
 (48.04416727525796, 172.06199406656083),
 (-26.923916420410734, -127.84049126667858),
 (-35.44725674224713, -12.900668561589754)]

In [37]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

18

In [78]:
# Create an endpoint URL for a city
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

for i, city_name in enumerate(cities):
# Build query URL
    city_name = city_name.replace(" ","+")
    query_url = url + "q=" + city_name + "&appid=" + api_key + "&units=" + units
    try:
        response = requests.get(query_url).json()
        country = response["sys"]["country"]
        date = response["dt"]
        date = datetime.utcfromtimestamp(response["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        lon = response["coord"]["lon"]
        lat = response["coord"]["lat"]
        max_temp = response["main"]["temp_max"]
        humidity =  response["main"]["humidity"]
        cloudiness = response["clouds"]["all"]
        wind_speed = response["wind"]["speed"]
        
        city_data.append({"City": city_name.title(),
                          "Lat": lat,
                          "Lng": lon,
                          "Max Temp": max_temp,
                          "Humidity": humidity,
                          "Cloudiness": cloudiness,
                          "Wind Speed": wind_speed,
                          "Country": country,
                          "Date": date})
    except:
        print(f"{city_name} weather not found.")
        continue

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
chuy
komsomolskiy
nouadhibou
rikitea
tuktoyaktuk
tasiilaq
punta+arenas
nhulunbuy
sovetskiy
hilo
kaitangata
alofi
bluff
acapulco
houma
georgetown
nikolskoye
jamestown
-----------------------------
Data Retrieval Complete      
-----------------------------


In [84]:
city_weather_df = pd.DataFrame(city_data)

In [85]:
city_weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Chuy,-33.6971,-53.4616,58.30,68,68,7.07,UY,2021-05-26 19:41:43
1,Komsomolskiy,40.4272,71.7189,71.02,36,64,4.70,UZ,2021-05-26 19:41:43
2,Nouadhibou,20.9310,-17.0347,67.98,73,12,21.85,MR,2021-05-26 19:41:01
3,Rikitea,-23.1203,-134.9692,76.48,71,82,21.47,PF,2021-05-26 19:41:43
4,Tuktoyaktuk,69.4541,-133.0374,30.20,74,75,11.50,CA,2021-05-26 19:41:44


In [86]:
column_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"]
city_weather_df = city_weather_df[column_order]
city_weather_df.head()

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Chuy,UY,2021-05-26 19:41:43,-33.6971,-53.4616,58.30,68,68,7.07
1,Komsomolskiy,UZ,2021-05-26 19:41:43,40.4272,71.7189,71.02,36,64,4.70
2,Nouadhibou,MR,2021-05-26 19:41:01,20.9310,-17.0347,67.98,73,12,21.85
3,Rikitea,PF,2021-05-26 19:41:43,-23.1203,-134.9692,76.48,71,82,21.47
4,Tuktoyaktuk,CA,2021-05-26 19:41:44,69.4541,-133.0374,30.20,74,75,11.50


In [ ]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_weather_df.to_csv(output_data_file, index_label="City_ID")